# Advanced Topics: Common Pitfalls and Dealing with Errors

**Welcome**

Learning Objectives:


* Learn how to read Python error messages
* Practice using Python error messages to track down problems in your code
* Know about some of the (non-error) mistakes that are easy to make in Python and learn ways to avoid them






## The basics of error messages

At various points throughout this workshop series, we've shown you examples of code that will cause errors. You may have noticed that attempting to run such code causes Python to print out a message, known as an *error message*, that explains exactly what the error was. As a reminder, let's look at the following piece of code that attempts to add an integer and a string without casting:




In [ ]:
print(1 + "2")

This error message contains two important parts. First, there is an arrow pointing to the line of code that caused the error:

```
----> 1 print(1 + "2")
```
And second, there is a description of what the error was:
```
TypeError: unsupported operand type(s) for +: 'int' and 'str'
```
While there is a little bit of technical jargon here, you can still get a sense of what the error is by looking at some keywords. We see that the error is called a "TypeError", which tells us that the error has something to do with *types*. Then, we see that the types "int" and "str" are specifically mentioned. We know what these are referring to in the line of code the arrow is pointing to, because there is only one int in that line (the 1) and only one str (the "2"). Putting all of this information together, we can guess that the way to fix this is to change one of the types using casting:

In [ ]:
print(1 + int("2"))

And indeed, that worked!

One important thing to notice is that error messages do not tell you exactly what you need to do to fix the error. This is because most of the time, there is more than one possible solution! For example, instead of casting the "2" to an int, we could have casted the 1 to a str:

In [ ]:
print(str(1) + "2")

As a result, reading an error message and using it to solve the error is not a completely straightforward process. It requires some knowledge to fill in the gaps. In this case, the error message only told us that int and str caused problems when we tried to use them together in that line of code. To finish solving the problem, we needed to apply our Python knowledge that we learned earlier in this workshop series, remembering that the "+" operator can either perform addition on two ints or concatenation on two strs.

## Exercise

The following code attempts to change the 5th item in a list of hometowns to "Ithaca", but it contains an error. What error message do you get? What should you do to fix it?

In [ ]:
hometowns = ["Brooklyn", "Atlanta", "Hampton", "Brentwood", "Lexington"]
hometowns[5] = "Ithaca"
print(hometowns)

## Functions and tracebacks

Error messages get more complicated when functions are involved. This is because an error can happen *inside* a function. When this happens, Python will illustrate this through what is known as a *traceback*, showing **all** relevant lines of code leading up to the eventual error, as shown in the below example:

In [ ]:
def addFive(x):
    print(x + 5)

addFive("10")

As we can see, there are now *two* arrows! The first one points to `addFive(10)`, which is the line where we call the function. The second one points to `print(x + 5)`, which is the part of the function where the TypeError happens. This information is extremely useful, because in fact both lines of code contributed to the error! The TypeError here happened for the same reason as the first example: we tried to add a string to an int. But the string came from a different line of code (the first arrow) than the actual addition (the second arrow).

Once again there are multiple possible solutions to the error, depending on what we, as the authors of the code, actually intended. If we intended addFive to be a function that only works on ints, then the problem is that we gave it a string when we called it, so the solution is to fix the line where we call the function so that we give it an int instead:

In [ ]:
def addFive(x):
    print(x + 5)

addFive(10)

On the other hand, if we intended addFive to be a function that can take non-int values, then the solution is to always cast the parameter x to an int:

In [ ]:
def addFive(x):
    print(int(x) + 5)

addFive("10")

Of course, we had multiple options here because addFive was a function that we wrote ourselves. But in practice, most code we write will use functions that *someone else* wrote - that is, functions from a package. Sometimes errors can arise when using a function that you imported from a package, and in this case the traceback might show you where in the package function the error happened, just like in our addFive example:

In [ ]:
import numpy as np
print(np.max([10, 3, 8, 21, "47", 50]))

Like in the addFive example, here the error happens because a line inside the np.max function does not work with the argument we gave when calling np.max (a list with ints and strs). But in this case, we did not write the np.max function ourselves, which means we cannot change anything inside the function - we have to obey the package author's intent! And in this case the package author's intent is that np.max cannot be used on lists that have mixed types. So the only solution we have is to change the code where we called the function, to give it an argument that is allowed, such as a list of only ints.

In [ ]:
import numpy as np
print(np.max([10, 3, 8, 21, 47, 50]))

One last thing to know about tracebacks is that they can get very "deep"; that is, you can get a traceback that points to *many* lines of code, not just one or two. This happens when you have a lot of functions calling other functions. Here's a modified version of our addFive example involving multiple function calls:

In [ ]:
netIDs = {"Jonathan": "jpc362", "Sam": "sd567"}

def addFive(x):
    print(x + 5)

def add_netid_number(netid):
    number = netid[-3:]
    addFive(number)

def add_to_name(name):
    netid = netIDs[name]
    add_netid_number(netid)

add_to_name("Jonathan")

As we can see, the resulting traceback has arrows pointing to 4 lines of code. Here's a walkthrough of what happened:

1.   We called the add_to_name function with the argument "Jonathan".
2.   add_to_name looked up Jonathan's netID, "jpc362", and then called add_netid_number with that string as the argument.
3.   add_netid_number extracted the number part of Jonathan's netID, which is "362" (but importantly, because this was done using string slicing, it is still a string!). It then called addFive with "362" as the argument.
4.   addFive tried to add the string "362" to the int 5, causing a TypeError.

Like before, the solution would be to cast the string "362" to an int, either by always casting the argument x in addFive, or by casting the variable number inside the add_netid_number function.



## Challenge

The code below, which attempts to compute the median age of all non-underaged participants in a dataset, involves a large number of function calls and eventually causes an error. Answer the following questions:

1.   What error occurs, and inside which function?
2.   What is the reason for the error (for example, in the addFive example, the reason for the error was that we tried to add two things of different types)
3.   How can the error be fixed? (There may be multiple valid solutions)

HINT: Remember that if you are ever unsure of the type of a variable, you can use the type() function to find out what it is.

In [ ]:
ages = [15, 23, 54, 17, 77, 42, 35, 16, 33, 72, 80, 22, 27, 45]

def get_middle_index(the_list):
    number_of_items = len(the_list)
    return number_of_items / 2

def get_nth_sorted_item(the_list, n):
    sorted_list = sorted(the_list) # note: the sorted() function returns a sorted version of the list
    return sorted_list[n]

def median_with_age_limit(list_of_ages, age_limit):
    valid_ages = []
    for age in list_of_ages:
        if age > age_limit:
            valid_ages.append(age)
    middle_index = get_middle_index(valid_ages)
    return get_nth_sorted_item(valid_ages, middle_index)

def median_adult_age(list_of_ages):
    age = median_with_age_limit(list_of_ages, 18)
    print(age)

median_adult_age(ages)

## Common pitfalls (and how to avoid them)

Thus far, we have discussed *errors* which are problems so severe that Python will be completely unable to run the affected code. As we have seen, when errors happen, you can use the error message to track down the exact problem and fix it. Error messages are not always straightforward, but they are better than nothing!

Unfortunately, you will also often find yourself facing problems that are *not* errors. That is, your code will run perfectly fine but it will not behave the way you were expecting. These are caused by *human* mistakes - for example, maybe you use the wrong comparison in an if statement, like using < instead of >. And because no actual error occurs, Python will not tell you anything is wrong (because from its perspective, **there is nothing wrong**), which can make these problems very difficult to track down! 

Solving these problems is more of an art than a science, so sadly there is no step-by-step guide we can offer for solving every single possible issue. But instead what we can offer is an overview of some common pitfalls that are easy to fall into when writing Python code. If you are not aware of these pitfalls, you can easily end up writing code that produces unexpected behavior! As such, if facing code that is not working the way you expect, one first step might be to check that you are not making any of these easy-to-make mistakes.

## Pitfall 1: The consequences of mutability
In Python, some types of objects are "mutable" and others are "immutable". "Immutable" means that the object cannot be changed after creation, "mutable" means that the object can be changed.

A good example to illustrate the difference is strings versus lists. Strings are immutable while lists are mutable. Therefore, even though strings and lists work similarly in terms of indexing, there are certain things you can do with lists that you cannot do with strings. You can change the items in a list, because lists are mutable:

In [ ]:
Hometown_list = ["Brooklyn", "Atlanta", "Hampton", "Brentwood", "Lexington"]
Hometown_list[0] = "Ithaca"
print('Hometown List is now:', Hometown_list)

By contrast, you cannot change the characters in a string the same way:

In [ ]:
mystring = "Donut"
mystring[0] = 'C' # this will cause an error

Other examples of mutable types include dictionaries and sets, while other examples of immutable types include tuples.

**Mutability has consequences!**

The fact that lists are mutable  means that any other variables pointing to a list will also be changed if that list gets changed. This can cause a lot of confusion if you're not expecting it! 

In the following example, the variables Hometown_list and new_list both refer to the same list value, and as a result when we change Hometown_list, new_list also gets changed.

In [ ]:
new_list = Hometown_list
print("original list: ", Hometown_list)
print("new list: ", new_list)
print()

Hometown_list[0] = "Jersey"
print("original list: ", Hometown_list)
print("new list: ", new_list)

Furthermore, mutability goes both ways! Continuing off the previous example, changing new_list will also change the original Hometown_list:

In [ ]:
new_list[0] = "Pasadena"
print("original list: ", Hometown_list)
print("new list: ", new_list)

**What if we don't want to change the original variable?**

Mutability can be extremely useful, since it lets you modify data throughout your code. But as we have seen, it can sometimes cause problems too. What if you actually want to make a new variable that doesn't affect the original one? In this case, to avoid the consequences of mutability, we would need to make a *copy* of the object.

For lists, slicing automatically makes a copy of the data:

In [ ]:
Hometown_list = ["Brooklyn", "Atlanta", "Hampton", "Brentwood", "Lexington"] # here's our original hometown list again...

derived_list = Hometown_list[0:5] # a trick for making a copy of a list: make a slice that contains all indices from the list!
print("derived_list is originally: ", derived_list)

Hometown_list[0] = "Pensacola"
print("changed Hometown_list: ", Hometown_list)



print("unchanged derived_list:", derived_list)

For other types of objects, you can use the `copy` package, which is a default package in Python, to easily make copies. Here's a demonstration with dictionaries:

In [ ]:
import copy

original_dict = {'a': 1, 'b': 42, 'c': 99}
new_dict = original_dict # using regular variable assignment, the new variable new_dict will be affected by changes to original_dict
copy_dict = copy.copy(original_dict) # using copy, the new variable copy_dict will NOT be affected by changes to original_dict

original_dict['a'] = -99
print("original_dict is now:", original_dict)
print("new_dict is also affected, because dicts are mutable:", new_dict)
print("copy_dict is NOT affected, because it is a copy:", copy_dict)

## Pitfall 2: Mutating iterators

So far, we have seen that lists are mutable and this can lead to confusing behavior. You may recall that a common use of lists is to use them as collections in a loop. So you might wonder: what happens if you modify a list *while* you're iterating on it?

To see why doing this can be a pitfall, consider the following example that modifies the last item in a list while it is looping. What will be the last thing that gets printed out: "ITH" or "SYR"?

In [ ]:
upstate_airports = ["ELM", "BUF", "SYR"]
for i in upstate_airports:
    upstate_airports[-1] = "ITH"
    print(i)

As we can see, "ITH" gets printed. This is because the change to the list takes effect immediately, even though the list is currently being used in a loop.

This can get even more confusing when you change the *number of items* in a list while you're looping over it. For example, suppose instead of changing the last item in the list, we deleted it. How many things would get printed out, 2 or 3?

In [ ]:
upstate_airports = ["ELM", "BUF", "SYR"]
for i in upstate_airports:
    del upstate_airports[-1]
    print(i)

Again, the change takes place immediately. This is something you need to be extremely careful about, because it means that if you keep making a list *longer* while you are iterating on it, your loop might never finish running! This is what's known as an *infinite loop*.

It is therefore a good idea to avoid modifying a list while you're iterating on it, unless absolutely necessary...and you should *never* add or remove items from the list while iterating on it!

## Pitfall 3: Scope and naming conflicts

**The scope** 

**Scope** refers to the part of the program where a name (variable or function) is accessible. Python has 4 types of scopes (global, local, non local, and built-in). We will focus on the first two:

**The Global Scope**

Global scope is the main body of the program. Any variables or functions we define here will be available to all code. In other words, if a variable is defined in the global score, modifying that variable anywhere in our code will impact that same variable wherever else it is found on the code. As you can imagine, this might get very messy! So we try to avoid using the global scope, which would be hard to debug later on. 

**The Local Scope**

Local scope is the breadth of a function. More specifically, every function has its own unique local scope. The variables defined inside a function are only available inside that function. In other words, they exist only when the function is executed. These variables are created each time we CALL the function, rather than when we define them. 

Let's see an example of using a variable in the global scope:

In [ ]:
country = "Argentina" #A global variable

def python_welcome(): #A function calling a global variable
    welcome = "Welcome to"
    print(welcome, country,"!")
python_welcome()

def nationality_name(name): #Another function calling a global variable
    print(name, "is from", country)
nationality_name("sam")

As we can see, we were able to access the variable "country" inside multiple functions. This is because "country" was defined outside the functions, and is therefore in global scope, which means that it can be accessed anywhere in our code.

And now let's see an example of using a variable in a local scope:

In [ ]:
def a_local_value(): #A function with a local variable in it
    value = 1 #A local variable
    print("The value defined is: ", value)
a_local_value()

What do you think would happen if I call a variable named inside a function, outside of this function?

Try the following code to test your guess:

In [ ]:
def a_local_value(): #A function with a local variable in it
    value = 1 #A local variable
print("The value defined is: ", value)

Sometimes we will need to call on a local variable as if it were global, or viceversa, that is, we will want to modify their behavior. 

The keyworks global and local will let us do that.

For example, the error in the prior code block, we could fix as follows:

In [ ]:
#Creating a variable called another_value within a function, but modifying its behavior with keyword global
def a_local_value(): 
    global another_value
    another_value = 3
    print("In Function value is: ", another_value)
a_local_value()

#Calling that variable from the global scope
print("Outside Function value is: ", another_value)

By using the global keyword, we were able to force the variable "another_value" to be in global scope, even though it was defined inside a function. Making it global allowed us to access it outside the function.

The pitfall you need to be very careful about is what happens when the *same* name appears in multiple scopes! For example, consider the following code which defines a variable called "my_name" in the global scope, then defines *another* variable *also* called "my_name" in a function's local scope. What do you think will be printed out, "Jonathan" or "Sam"?

In [ ]:
my_name = "Jonathan" # Defining my_name in global scope

def say_my_name():
    my_name = "Sam" # Defining my_name in local scope
    print(my_name)

say_my_name() # What gets printed?

As we can see, "Sam" got printed. This is because the local scope takes priority. Also, because the local version of my_name is different from the global one, accessing my_name outside the function will *still* print "Jonathan":

In [ ]:
print(my_name)

If we had actually wanted to change the global variable my_name inside the function, we would need to use the global keyword as we discussed before. But remember, what this is doing is telling Python to treat my_name as the global version, so any changes we make this way will also affect things outside the function!

In [ ]:
my_name = "Jonathan"

def say_my_name():
    global my_name # force the function to access the global version of my_name
    my_name = "Sam"
    print("Inside function:", my_name)

say_my_name()
print("Outside function:", my_name)

You can see how this will get extremely confusing! To avoid confusion, try to do the following when you write code:

*   Avoid defining variables in the global scope
*   If you must create global variables, give them unique names; you want to avoid using the same names for local variables!

